In [1]:
import numpy as np
from numpy import deg2rad, sqrt, pi, sin, cos
from numpy.linalg import norm
import matplotlib.pyplot as plt
from scipy.optimize import newton
from scipy.spatial.transform import Rotation

In [2]:
MU = 3.986004418E5
C = 300 # km/ms

# Exercício 1 (Parametrização das órbitas)

## Item a

Uma parametrização é o processo de encontrar equações paramétricas que descrevem uma curva ou superfície. Expressar essas formas em termos de parâmetros. Uma curva é a imagem de uma função $\gamma: I\subseteq \mathbb{R}^n \to \mathbb{R}^m$.

Nesse primeiro caso, quero encontrar a parametrização cartesiana de uma curva que representa a órbita do satélite em torno da terra.

$$\gamma(t) = \left(x(t), y(t)\right)$$

Da definição de coordenadas polares,

$$\begin{cases}
x(t) = r(t)\cos(\theta(t))\\
y(t) = r(t)\sin(\theta(t))\end{cases}$$

Sendo que $r(t)$ é a distância entre a origem e o ponto da curva e $\theta(t)$ é o ângulo marcado em sentido antihorário com a parte positiva do eixo $x$. 

Portanto, como o centro da terra é colocado na origem $r(t)$ (distância entre a terra e o satélite) é mesma distância definida nas coordenadas polares. De maneira similar, os ângulos são definidos da mesma maneira $\theta(t) = \nu(t)$.

$$\begin{cases}
x(t) = r(t)\cos(\nu(t))\\
y(t) = r(t)\sin(\nu(t))\end{cases}$$

Em seguida, definido o círculo da figura. 

Caso o sistema de coordenadas fosse o centro do círculo, a parametrização seria a conhecida parametrização da elipse:

$$\begin{cases}
x(t) = a\cos E(t)\\
y(t) = b\sin E(t)\end{cases}$$

No entanto, o sistema de coordenadas está deslocado no eixo $x$ e centrado no foco da elipse. Isso é resultado de uma das Leis de Kepler que diz que "a órbita de um corpo é uma elipse com o atrator localizado em um de seus focos". 

Da figura, podemos relacionar as distâncias no eixo $x$ com a seguinte equação

$$a\cos E = ae + r\cos \nu$$

![](imgs/circulo_ecentrico.svg)

Mais informações sobre a anomalia excêntrica: [Orbital Mechanics, Bryan Weber](https://orbital-mechanics.space/time-since-periapsis-and-keplers-equation/elliptical-orbits.html#eccentric-anomaly).

Sabendo que $ae$ é a distância entre o centro do círculo e a posição do planeta (distância do centro ao foco). 

---

**Demonstração** 

Uma característica definidora da Elipse é que *soma das distâncias de um ponto para cada um de seus focos é constante*. Assim, quando o ponto está no eixo $x$, podemos mostrar que essa constante é igual a $2a$. Além disso, quando o ponto está a $90^\circ$ graus do eixo $x$ positivo, as duas distância são iguais e, portanto, valem $a$. Nesse esquema, denotando como distância do centro ao foco como $c$, pode ser obtida a relação

$$\text{(dist. centro ao foco)} = c = \sqrt{a^2-b^2}$$

A excentricidade é a razão entre a distância do centro até o foco e o semieixo maior.

$$e:= \frac{c}{a} = \sqrt{1 - \frac{b^2}{a^2}}$$

Portanto, dada a excentricidade $e$ e o semieixo maior $a$ a distância do centro ao foco é

$$c = ae$$

---

A igualdade dos comprimentos acima, indica que

$$x = r\cos\nu = \boxed{a(\cos E - e)}$$

A coordenada $y$ da elipse com centro coincidente ao centro do círculo é dada por $y = b\sin E$. No entanto, a mesma coisa vale para o sistema de coordenadas centrado no foco (já que não existe deslocamento vertical). Por fim, da definição de excentricidade, $b = a \sqrt{1-e^2}$

$$y = \boxed{a\sqrt{1-e^2}\sin E}$$

## Item b

(Sobre o período orbital)

Da segunda Lei de Kepler,

$$\Delta A = \frac{h}{2}\Delta t$$

Onde $h$ é o momento angular. Em órbitas elípticas, ele é dado por

$$h = \sqrt{a\mu(1-e^2)}$$

Para a área total, teremos o período

$$\pi ab = \frac{h}{2}T \Leftrightarrow T = 2\pi\sqrt{\frac{a^3}{\mu}}$$

(Sobre equações transcendentes)

São equações que envolvem uma função transcendente e não apresentam solução analítica. 

No caso da equação de Kepler, não existe solução algébrica para $E$. Para resolver esse tipo de equação, é necessário o uso de métodos iterativos ou de expansões em série.

Lagrange e Bessel desenvolveram séries que calculam o valor de $E$ dado um valor de $M$.

Vide: [Wikipedia](https://en.wikipedia.org/wiki/Kepler's_equation#Equation) e [Bryan Weber](https://orbital-mechanics.space/time-since-periapsis-and-keplers-equation/elliptical-orbits.html#infinite-series-solutions-of-kepler-s-equation)

No entanto, vamos usar o método de Newton que consiste em definir uma função $f(x)$ tal que $E$ seja uma raíz dessa função. Nesse caso,

$$f(E) = E - e \sin E - M = 0$$

O método está baseado em encontrar a raíz de $f$ usando sua derivada. Em especial, a cada passo iterativo do método, fazemos uma aproximação linear da função e encontramos a raíz dessa reta (interseção com o eixo $x$).

$$E_{k+1} = E_k - \frac{f(E_k)}{f'(E_{k})}$$

Em especial, vamos usar a função `newton` da biblioteca `scipy.optimize`.

Algumas noções de convergência e estabilidade numérica devem acompanhadar esse processo. Mas não é o foco do projeto. 

Aqui, usamos um palpite inicial ($E_0$) igual a anomalia média.

In [3]:
def perifocalPos(tp, a, e, mu = MU):
    
    def kepler(E, M, e):
        # Equação de Kepler na forma: f(E) = 0
        return E - e * sin(E) - M

    def dKepler(E, M, e):
        # Derivada da Equação de Kepler f'(E)
        return 1 - e * cos(E)

    tp = np.atleast_1d(tp)
    
    T = 2 * pi / sqrt(mu) * a**(3 / 2) # Período Orbital (seg)
    M = (2*pi/T * tp) % (2*pi) # Anomalia Média

    # Resolve a Equação de Kepler para a Anomalia Excêntrica
    E0 = M
    E = newton(func=kepler, x0=E0, fprime=dKepler, args=(M, e))
    
    # Resposta do item a)
    x = a*cos(E) - a*e
    y = a*sqrt(1-e**2)*sin(E)
    
    return np.array([x, y])

## Item c

Matrizes de rotação em torno de cada um dos eixos são dadas por

$$R_x(\theta) = \begin{bmatrix}
1 & 0 & 0 \\
0 & \cos \theta & -\sin \theta\\
0 & \sin \theta & \cos \theta
\end{bmatrix}$$

$$R_y(\theta) = \begin{bmatrix}
\cos \theta & 0 & \sin \theta\\
0 & 1 & 0\\
-\sin \theta & 0 & \cos \theta
\end{bmatrix}$$


$$R_z(\theta) = \begin{bmatrix}
\cos \theta & -\sin \theta & 0\\
\sin \theta & \cos \theta & 0\\
0 & 0 & 1
\end{bmatrix}$$

Rotações em sequência podem ser obtidas pela composição de transformações (multiplicação das matrizes). Por exemplo, a rotação em torno de $z$, seguida de uma rotação em torno de $x$ e, por fim, uma rotação em torno de $z$ é dada por:

$$R_zR_xR_z\mathbf{v}$$

Sendo $\mathbf{v}$ um ponto qualquer no espaço. Observe que a ordem é da direita para esquerda.

Na ordem dada pelo enunciado, a matriz de rotação resultante é

$$R_z(\Omega)R_x(i)R_z(\omega)$$

https://orbital-mechanics.space/classical-orbital-elements/orbital-elements-and-the-state-vector.html

In [62]:
def Rx(t):
    return np.array([
        [1, 0, 0],
        [0, cos(t), -sin(t)],
        [0, sin(t), cos(t)],
    ])

def Ry(t):
    return np.array([
        [cos(t), 0, sin(t)],
        [0, 1, 0],
        [-sin(t), 0, cos(t)],
    ])

def Rz(t):
    return np.array([
        [cos(t), -sin(t), 0],
        [sin(t), cos(t), 0],
        [0, 0, 1],
    ])

In [63]:
def posVec(tp, a, e, w, i, o, mu = MU):
    pPos = perifocalPos(tp, a, e, mu)
    
    perifocal3D = np.vstack([pPos, np.zeros(pPos.shape[1])])
    
    # Realiza as três rotações
    w, i, o = tuple(deg2rad(np.array([w, i, o])))
    R = Rz(o) @ Rx(i) @ Rz(w)
    pos = R @ perifocal3D

    return pos

## Teste

In [64]:
# Dados
a = np.array([15300, 16100, 17800, 16400]) # km
e = np.array([.41, .342, .235, .3725])
w = np.array([60, 10, 30, 60]) # deg
i = np.array([30, 30, 0, 20]) # deg
o = np.array([0, 40, 40, 40]) # deg
tp = np.array([4708.5603, 5082.6453, 5908.5511, 5225.3666])
TOA = np.array([60000] * 4)
TOT = np.array([13581.1080927 , 19719.32768037, 11757.73393255, 20172.46081236])

In [65]:
satPos = []
for k in range(4):
    pos = posVec(tp[k], a[k], e[k], w[k], i[k], o[k])
    satPos.append(pos)

satPos = np.array(satPos).reshape([4, 3])
satPos

array([[-17198.94636766,  -3357.8884269 ,  -1938.67778718],
       [-16764.51326576,   -188.27453647,   6138.26955927],
       [-18646.04514963,  -1962.47472564,      0.        ],
       [-12159.76207073, -13896.76819502,  -1029.81652816]])

# Exercício 2

## Item a


$$TOF = TOA - TOT$$

$$\rho = c \cdot TOF$$

$$c = 3\times 10^8\ \text{m/s}$$

In [66]:
TOF = TOA - TOT
TOF

array([46418.8919073 , 40280.67231963, 48242.26606745, 39827.53918764])

In [67]:
rho = TOF*C/1000
rho

array([13925.66757219, 12084.20169589, 14472.67982024, 11948.26175629])

## Item b

Sistema não linear, pois as equações não são combinações lineares das variáveis $(x,y,z)$

$$ (x - x)^2 + (y - y_i)^2 + (z - z_i)^2 = \rho_i^2$$

Não podemos usar os métodos da álgebra linear para resolver o sistema.

## Item c

$$J(x,y,z) = \frac{1}{2}\sum_i\left(\sqrt{(x - x_i)^2 + (y - y_i)^2 + (z - z_i)^2} - \rho_i \right)^2$$

$J$ é zero quando acertamos a posição do receptor. Para qualquer outro valor, o valor de $J$ será maior. Dessa forma, a posição do drone é um mínimo global de $J$.


## Item d

Wikipedia:

Gradient descent is a method for unconstrained mathematical optimization. It is a first-order iterative algorithm for finding a local minimum of a differentiable multivariate function.

The idea is to take repeated steps in the opposite direction of the gradient (or approximate gradient) of the function at the current point, because this is the direction of steepest descent. Conversely, stepping in the direction of the gradient will lead to a local maximum of that function; the procedure is then known as gradient ascent. It is particularly useful in machine learning for minimizing the cost or loss function.

$$\mathbf{x}_{k+1} = \mathbf{x}_k - \alpha \nabla J(\mathbf{x}_k)$$

Levando em conta o **gradiente descendente com momento**, defino a velocidade como

$$\mathbf{v}_{k+1} = \beta\mathbf{v}_k - \alpha \nabla J(\mathbf{x}_k)$$

$$\mathbf{x}_{k+1} = \mathbf{x}_k + \mathbf{v}_{k+1}$$

Mais sobre:

- [Redução da perda: taxa de aprendizado, Google for Devs](https://developers.google.com/machine-learning/crash-course/reducing-loss/learning-rate?hl=pt-br)
- [Why Momentum Really Works](https://distill.pub/2017/momentum/)

---

$$\partial_x J = \sum_i(x-x_i)\frac{\sqrt{(x - x_i)^2 + (y - y_i)^2 + (z - z_i)^2} - \rho_i}{ \sqrt{(x - x_i)^2 + (y - y_i)^2 + (z - z_i)^2}}$$

Para encurtar,

$$\hat \rho_i := \sqrt{(x - x_i)^2 + (y - y_i)^2 + (z - z_i)^2}$$

$$\partial_x J = \sum_i(x-x_i)\frac{\hat \rho_i - \rho_i}{\hat \rho_i}$$

Por simetria, obtém-se as outras derivadas que compõem o gradiente

$$\partial_y J = \sum_i(y-y_i)\frac{\hat \rho_i - \rho_i}{\hat \rho_i}$$
$$\partial_z J = \sum_i(z-z_i)\frac{\hat \rho_i - \rho_i}{\hat \rho_i}$$

$$\nabla J = \begin{bmatrix}
    \partial_x J \\
    \partial_y J \\
    \partial_z J \\
\end{bmatrix}$$

Em formato de matriz, tem-se

$$\mathbf{x} = (x, y, z)$$

$$\mathbf{R} = \begin{bmatrix}
    x_1 & y_1 & z_1 \\
    x_2 & y_2 & z_2 \\
    & \vdots & \\
    x_n & y_n & z_n \\
\end{bmatrix}$$

$$\mathbf{d} = \mathbf{x} - \mathbf{R} = \begin{bmatrix}
    x - x_1 & y - y_1 & z - z_1 \\
    x - x_2 & y - y_2 & z - z_2 \\
    & \vdots & \\
    x - x_n & y - y_n & z - z_n \\
\end{bmatrix}_{(n\times 3)}$$

$$\boldsymbol{\rho} = \begin{bmatrix}
    \rho_1 \\ \rho_2 \\ \vdots \\ \rho_n
\end{bmatrix},\quad
\boldsymbol{\hat \rho} = \begin{bmatrix}
    \hat\rho_1 \\ \hat\rho_2 \\ \vdots \\ \hat\rho_n
\end{bmatrix},\quad
\frac{\boldsymbol{\hat \rho}  - \boldsymbol{\rho}}{\boldsymbol{\hat \rho} } = \begin{bmatrix}
    (\hat \rho_1 - \rho_1)/\hat \rho_1 \\ 
    (\hat \rho_2 - \rho_2)/\hat \rho_2 \\ 
    \vdots \\ 
    (\hat \rho_n - \rho_n)/\hat \rho_n
\end{bmatrix}_{(n\times 1)}$$

Logo, o vetor gradiente é

$$\nabla J = \mathbf{d}^T \left(\frac{\boldsymbol{\hat \rho}  - \boldsymbol{\rho}}{\boldsymbol{\hat \rho} }\right) = \mathbf{d}^T \left(1 - \frac{\boldsymbol{\rho}}{\boldsymbol{\hat \rho} }\right) $$

In [72]:
def grad(x):
    # satPos, rho são variáveis globais
    d = x - satPos
    rhoHat = norm(d, axis=1)
    return d.T @ (1 - rho/rhoHat)

In [69]:
grad(np.array([-6420., -6432., 6325.]))

array([-8.03850507e-10,  2.15552320e-10, -1.92621096e-10])

In [70]:
def gradient_descent(grad, x0, Nsteps=200, learningRate=0.6, momentum=0.7):
    x = x0
    vel = np.array([[0, 0, 0]], dtype='float64')
    
    for i in range(Nsteps):
        g = grad(x)
        
        vel = momentum * vel - learningRate * g
        x += vel
    
    return x

In [71]:
x0 = np.array([[0, 0, 0]], dtype='float64')

gradient_descent(grad, x0, Nsteps=200)

array([[-6420., -6432.,  6325.]])